<a href="https://colab.research.google.com/github/AhmedGS/deep_learning_keras/blob/master/Cats_vs_Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Jun  5 20:38:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!pip install tf-nightly

     |████████████████████████████████| 526.2MB 31kB/s 
     |████████████████████████████████| 460kB 57.3MB/s 
     |████████████████████████████████| 3.0MB 49.3MB/s 


In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  786M  100  786M    0     0  51.7M      0  0:00:15  0:00:15 --:--:-- 26.8M


In [5]:
!unzip -q kagglecatsanddogs_3367a.zip
!ls

 kagglecatsanddogs_3367a.zip   PetImages        sample_data
'MSR-LA - 3467.docx'	      'readme[1].txt'


In [9]:
# !rm kagglecatsanddogs_3367a.zip
!ls PetImages

Cat  Dog


In [10]:
import os

num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

Deleted 0 images


In [0]:
class Model:

  def __init__(self, data_path, n_classes=2, image_size=(180, 180), batch_size=32):
    self.data_path = data_path
    self.image_size = image_size
    self.batch_size = batch_size
    self.get_data()
    # self.data_augmentation()
    self.make_model(n_classes)

  def get_data(self, rand_seed=1337):
    self.train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=self.image_size,
    batch_size=self.batch_size,
    )
    self.val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=self.image_size,
    batch_size=self.batch_size,
    )

  # def get_data_generators(self, rand_seed=1337):
  #   self.train_generator = self.train_datagen.flow_from_directory(
  #       self.data_path,  # This is the source directory for training images
  #       target_size=self.image_size,  # All images will be resized to 200 x 200
  #       batch_size=self.batch_size,
  #       # Specify the classes explicitly
  #       classes = ['Cat','Dog'],
  #       subset="training",
  #       shuffle=True,
  #       class_mode='binary')
    
  #   self.val_generator = self.val_datagen.flow_from_directory(
  #       self.data_path,
  #       target_size=self.image_size,  # All images will be resized to 200 x 200
  #       batch_size=self.batch_size,
  #       # Specify the classes explicitly
  #       classes = ['Cat','Dog'],
  #       subset="validation",
  #       shuffle=True,
  #       class_mode='binary'
  #   )

  def data_augmentation(self):

    self.train_datagen = ImageDataGenerator(
        rescale=1./255,
        # rotation_range=30,
        # width_shift_range=0.2,
        # # height_shift_range=0.2,
        # shear_range=0.2,
        # zoom_range=0.2,
        # horizontal_flip=True,
        validation_split=0.2,
        # fill_mode='nearest'
        )
    self.val_datagen = ImageDataGenerator(
        rescale=1./255
        )


  def evaluate_model(self):
    print(self.model.evaluate(self.val_ds))

  def make_model(self, num_classes=2):

    data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
    )
    
    input_shape = self.image_size + (3,)
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)
    # x = inputs

    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    self.model = keras.Model(inputs, outputs)


  def train(self, n_epoch=50):

    self.model.compile(
      optimizer=keras.optimizers.RMSprop(lr=0.001),
      loss="binary_crossentropy",
      metrics=["accuracy"],
      )
    
    total_sample = 23410 * 0.8
    # callbacks = [keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),]

    self.history = self.model.fit(
        self.train_ds, 
        steps_per_epoch=int(total_sample/self.batch_size),  
        epochs=n_epoch,
        # callbacks=callbacks,
        validation_data=self.val_ds,
        verbose=1)
    
   

    



In [31]:
kmodel = Model('PetImages')
# kmodel.get_data()

Found 23410 files belonging to 2 classes.
Using 18728 files for training.
Found 23410 files belonging to 2 classes.
Using 4682 files for validation.


In [32]:
kmodel.train(n_epoch=1)

585/585 [==============================] - 144s 246ms/step - loss: 0.6640 - accuracy: 0.6330 - val_loss: 0.6046 - val_accuracy: 0.6805


In [33]:
kmodel.evaluate_model()

147/147 [==============================] - 11s 77ms/step - loss: 0.6046 - accuracy: 0.6805
[0.6046444773674011, 0.6804784536361694]


In [0]:
# kmodel.model.summary()

In [0]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# for images, labels in kmodel.train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(int(labels[i]))
#         plt.axis("off")